# **1. Llamada a la API de OpenAI**

Para cada búsqueda realizada de los distintos empleos en Madrid hemos obtenido varias ofertas de trabajo, de las cuales tenemos su correspondiente descripción. Para cada descripción se pretende obtener cúanto se ajusta dicha oferta al puesto para el cual se realizó la búsqueda, por ejemplo cuanto se ajusta la descripción a un puesto de Data Science, cuál es el nivel de inglés si se pidiera, los años de experiencia si se pidieran, las hard skill necesarias y las hard skills deseables.

Para llevar a cabo esta tarea usaremos IA generativa a través de la API de OpenAI, la cual se encargará de llevar a cabo la labor de extraer de cada oferta la información deseada.

In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import ast
import pandas as pd
import pickle
import json
import os

import sys
sys.path.append("../")

from src import funciones_transformacion as f_transform
from src import funciones_bbdd as f_bbdd

load_dotenv()

True

El primer paso es obtener el listado de los archivos para ambas plataformas en las que se encuentren las descripciones de las ofertas de trabajo.

In [8]:
lista_ofertas=[]

# Files de infojobs
folder_path_infojobs = "../scraping_infojobs/datos_limpios/"
files_infojobs = os.listdir(folder_path_infojobs)
[lista_ofertas.append(folder_path_infojobs + file) for file in files_infojobs]

# Files de linkedin
folder_path_linkedin = "../scraping_linkedin/data/datos_descripcion_ofertas/"
files_linkedin = os.listdir(folder_path_linkedin)
[lista_ofertas.append(folder_path_linkedin + file) for file in files_linkedin]

lista_ofertas

['../scraping_infojobs/datos_limpios/infojobs_data_analyst.csv',
 '../scraping_infojobs/datos_limpios/infojobs_data_engineer.csv',
 '../scraping_infojobs/datos_limpios/infojobs_data_science.csv',
 '../scraping_linkedin/data/datos_descripcion_ofertas/linkedin_data_analyst.csv',
 '../scraping_linkedin/data/datos_descripcion_ofertas/linkedin_data_engineer.csv',
 '../scraping_linkedin/data/datos_descripcion_ofertas/linkedin_data_science.csv']

El siguiente función de `extracion_datos_descripciones` se encargá de extraer de la descripción los años de experiencia que piden, el nivel de inglés, las hard skills necesarias, las hard skill deseables y un porcentaje de ajuste. Este porcetaje representa cuanto se ajusta la descripción de la oferta en cuanto al puesto para el que se realizó la busqueda, por ejemplo, Data analyst. Todos estos datos se añadirán a un json que se irá almacenando en la carpeta `data` en este mismo directorio.

In [9]:
f_transform.extracion_datos_descripciones(lista_ofertas)

True
Analizando el archivo -infojobs_data_analyst.csv- de la categoria: Data analyst, con shape: (108, 9)
Analizando el archivo -infojobs_data_engineer.csv- de la categoria: Data engineer, con shape: (73, 9)
Analizando el archivo -infojobs_data_science.csv- de la categoria: Data science, con shape: (39, 9)
Analizando el archivo -linkedin_data_analyst.csv- de la categoria: Data analyst, con shape: (934, 8)
Analizando el archivo -linkedin_data_engineer.csv- de la categoria: Data engineer, con shape: (948, 8)
Analizando el archivo -linkedin_data_science.csv- de la categoria: Data science, con shape: (745, 8)


_________________________________________________

## **Limpieza de los JSON**

A través de la API se han generado para cada oferta una lista de hard skills necesarias y deseables, sin embargo, hay que repasar esta lista para quedarnos únicamente con las hard skills y limpiar algunas soft skills que se han colado.

In [83]:
folder_path = "data/response_openai/"
df_files = os.listdir(folder_path)
skills = []

for file in df_files:
    print(f"Extrayendo skills del df {file} ...")
    df_data_transform = pd.read_json(folder_path + file)
    df_expanded = pd.json_normalize(df_data_transform['transformacion'])
    df_final = pd.concat([df_data_transform.drop(columns=['transformacion']), df_expanded], axis=1)

    for i in range(df_final.shape[0]):
        skills += df_final["skills_necesarias"].iloc[i]
        skills += df_final["skills_valoradas"].iloc[i]

set_skills = set(skills)
print(f"\nCompletado! Hemos obtenido un total de {len(set_skills)} skills")


Extrayendo skills del df transformed_infojobs_data_analyst.json ...
Extrayendo skills del df transformed_infojobs_data_engineer.json ...
Extrayendo skills del df transformed_infojobs_data_science.json ...
Extrayendo skills del df transformed_linkedin_data_analyst.json ...
Extrayendo skills del df transformed_linkedin_data_engineer.json ...
Extrayendo skills del df transformed_linkedin_data_science.json ...

Completado! Hemos obtenido un total de 3210 skills


Una vez tenemos la lista de todas las skills recopiladas para todas las ofertas filtramos de nuevo con la API de OpenAI cuales son relamente las que nos interesan.

In [84]:
lista_skills = list(set_skills)
lista_respuesta_skills = f_transform.extraer_hard_skills(lista_skills)

True


In [90]:
if lista_respuesta_skills.startswith("```python\n") and lista_respuesta_skills.endswith("\n```"):
        respuesta = lista_respuesta_skills[10:-4]
lista_resultado = ast.literal_eval(respuesta)
lista_hard_skills = list(set(lista_resultado))

with open('data/lista_hard_skills.pkl', 'wb') as archivo:
    pickle.dump(lista_hard_skills, archivo)

print(f"Hemos reducido las hard skills a un total de {len(lista_hard_skills)}")

Hemos reducido las hard skills a un total de 363


### Nuevo filtrado de las skills

Una vez hemos recopilado todas las skills de todas las ofertas y hemos extraido una lista de las verdaderamente relevantes el siguiente paso es ir recorriendo todas las ofertas y ver si tanto las skills necesarias como las valoradas se encuentran en dicha lista. 

Comprobaremos para cada oferta si sus skills están en la lista, o si se parecen a alguna skill de la lista devolviendo asi un diccionario de conversion, las que pasen en filtro se quedarán y el resto serán eliminadas. Guardaremos nuevos archivos de ofertas con las skills finales que se almacenarán en `data/skills_filtradas`.

In [208]:
diccionario_conversion_general = f_transform.filtracion_hard_skills()

100%|██████████| 200/200 [00:10<00:00, 18.41it/s]


In [11]:
#diccionario_conversion_general

____________________________________

# **2. Carga de datos en MongoDB**

Una vez hemos limpiado las skills de todas las ofertas de trabajo y las hemos almacenado en los correspondientes `json` el siguiente paso es subir todos estos datos a una base de datos en la nube. En este caso al ser tablas no relacionales hemos escogido MongoDB y los datos se subiran a MongoDB Atlas.

In [10]:
f_bbdd.cargar_datos_mongo()

Pinged your deployment. You successfully connected to MongoDB!
Insertando file: filtered_transformed_infojobs_data_analyst.json
Insertando file: filtered_transformed_infojobs_data_engineer.json
Insertando file: filtered_transformed_infojobs_data_science.json
Insertando file: filtered_transformed_linkedin_data_analyst.json
Insertando file: filtered_transformed_linkedin_data_engineer.json
Insertando file: filtered_transformed_linkedin_data_science.json
---- Insercion en la base de datos finalizada ----
